In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from pandas.plotting import lag_plot

# Load the data
data = pd.read_csv("C:/Users/sa3id/spiced/CS/convergence_oracle/Playground_SB/Topified_vectorized_Science1900_2023/web/pages/_forecasted_predicted_actual_Science_1900_2023__202310141011.csv")
data.head(25)


In [ ]:
data['value'].unique()

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# from ipywidgets import interact, widgets

# # Group the data by topic
# grouped_data = data.groupby('topic_list')

# # Function to create and display a plot for a specific topic
# def plot_topic(topic):
#     plt.figure(figsize=(12, 6))
#     topic_data = grouped_data.get_group(topic)
#     plt.plot(topic_data['date'], topic_data['quantity'])
#     plt.xlabel('Date')
#     plt.ylabel('Quantity')
#     plt.title(f'Quantity Over Time for Topic: {topic}')
#     plt.grid(True)
#     plt.show()

# # Create a dropdown widget with topic options
# topic_dropdown = widgets.Dropdown(
#     options=data['topic_list'].unique(),
#     description='Select Topic:',
# )

# # Create an interactive widget to select a topic
# interact(plot_topic, topic=topic_dropdown)


In [ ]:

lag_plot(data['quantity'])
plt.title('Lag Plot')
plt.show()


In [ ]:

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(data['quantity'], bins=20, density=True, alpha=0.6, color='b')
plt.title('Histogram of Quantity')

plt.subplot(1, 2, 2)
stats.probplot(data['quantity'], dist="norm", plot=plt)
plt.title('Probability Plot')
plt.show()


another test


In [ ]:
# Pivot the data to create separate columns for each value
pivoted_data = pd.pivot_table(data, index=['date', 'topic_list','metric', 'RMSE', ], columns='value', values='quantity', aggfunc='first').reset_index()

# Rename the columns for better clarity
pivoted_data.columns.name = None  # Remove the columns name
pivoted_data.rename(columns={'Actual_Test': 'Actual', 'Actual_Train': 'Train', 'Forecasted': 'Forecasted', 'Predicted_Test': 'Predicted'}, inplace=True)

# Optional: Replace NaN values with zeros
#pivoted_data.fillna(0, inplace=True)

# Now, pivoted_data contains separate columns for each value
#pivoted_data.to_csv('pivoted_data.csv', index=False)


In [ ]:
pivoted_data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# Load your data
data = pivoted_data

# Function to create and display a plot for a specific topic
def plot_topic(selected_topic):
    plt.figure(figsize=(12, 6))
    topic_data = pivoted_data[pivoted_data['topic_list'] == selected_topic]
    plt.plot(topic_data['date'], topic_data['Train'], label='Train')
    plt.plot(topic_data['date'], topic_data['Actual'], label='Actual')
    plt.plot(topic_data['date'], topic_data['Forecasted'], label='Forecasted')
    plt.plot(topic_data['date'], topic_data['Predicted'], label='Predicted')
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.title(f'Values Over Time for Topic: {selected_topic}')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create a dropdown widget with topic options
topic_dropdown = widgets.Dropdown(
    options=pivoted_data['topic_list'].unique(),
    description='Select Topic:',
)

# Create an interactive widget to select a topic
interact(plot_topic, selected_topic=topic_dropdown)


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import io
import PIL
import base64

# Load your data
df = pivoted_data

# Create a Plotly figure
fig = go.Figure()

# Iterate through unique topics to create traces for each topic
for topic in df['topic_list'].unique():
    topic_data = df[df['topic_list'] == topic]
    
    # Use Matplotlib to add Train data
    plt.figure(figsize=(12, 6))
    plt.plot(topic_data['date'], topic_data['Train'], label=f'Train - {topic}')
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.title(f'Train Values Over Time for Topic: {topic}')
    plt.legend()

    # Save the Matplotlib plot as an image and convert it to a data URI
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img = PIL.Image.open(buf)
    img_data = io.BytesIO()
    img.save(img_data, format="PNG")
    img_uri = "data:image/png;base64," + base64.b64encode(img_data.getvalue()).decode()

    # Close the Matplotlib figure
    plt.close()

    # Add traces for Actual, Predicted, and Forecasted values
    fig.add_trace(go.Scatter(x=topic_data['date'], y=topic_data['Actual'], mode='lines', name=f'Actual - {topic}'))
    fig.add_trace(go.Scatter(x=topic_data['date'], y=topic_data['Predicted'], mode='lines', name=f'Predicted - {topic}'))
    fig.add_trace(go.Scatter(x=topic_data['date'], y=topic_data['Forecasted'], mode='lines', name=f'Forecasted - {topic}'))

    # Add the image to the Plotly figure using the data URI as the 'source' property
    fig.add_layout_image(
        x=0,
        y=0.25,
        xref="paper",
        yref="paper",
        xanchor="left",
        yanchor="bottom",
        source=img_uri,
        opacity=1,
        layer="below"
    )

# Set layout properties
fig.update_layout(
    title='Actual vs. Predicted vs. Forecasted by Topic',
    xaxis_title='Date',
    yaxis_title='Quantity',
    showlegend=True,
    xaxis_rangeslider_visible=True
)

# Add a dropdown for topic selection
buttons = [
    {
        'method': 'restyle',
        'label': topic,
        'args': [{'visible': [topic == f'Actual - {t}' or topic == f'Predicted - {t}' or topic == f'Forecasted - {t}' for t in df['topic_list']]}]
    }
    for topic in df['topic_list'].unique()
]

fig.update_layout(
    updatemenus=[
        {
            'buttons': buttons,
            'direction': 'down',
            'showactive': True,
            'x': 0.1,
            'xanchor': 'left',
            'y': 1.1,
            'yanchor': 'top'
        }
    ]
)

# Show the plot
fig.show()


In [ ]:
# import pandas as pd
# import plotly.graph_objects as go
# import matplotlib.pyplot as plt
# import numpy as np
# import io
# import PIL
# import base64
# from ipywidgets import widgets, interactive
# import ipywidgets as widgets
# import IPython.display as display

# # Load your data
# df = pivoted_data

# # Create a Plotly figure
# fig = go.Figure()

# # Function to update the Plotly figure based on the selected topic
# def update_plot(selected_topic):
#     fig.data = []  # Clear existing traces
#     for topic in df['topic_list'].unique():
#         if topic == selected_topic:
#             topic_data = df[df['topic_list'] == topic]
            
#             # Add traces for Actual, Predicted, and Forecasted values
#             fig.add_trace(go.Scatter(x=topic_data['date'], y=topic_data['Actual'], mode='lines', name=f'Actual - {topic}'))
#             fig.add_trace(go.Scatter(x=topic_data['date'], y=topic_data['Predicted'], mode='lines', name=f'Predicted - {topic}'))
#             fig.add_trace(go.Scatter(x=topic_data['date'], y=topic_data['Forecasted'], mode='lines', name=f'Forecasted - {topic}'))

#             # Use Matplotlib to add Train data
#             plt.figure(figsize=(12, 6))
#             plt.plot(topic_data['date'], topic_data['Train'], label=f'Train - {topic}')
#             plt.xlabel('Date')
#             plt.ylabel('Value')
#             plt.title(f'Train Values Over Time for Topic: {topic}')
#             plt.legend()
            
#             # Save the Matplotlib plot as an image and convert it to a data URI
#             buf = io.BytesIO()
#             plt.savefig(buf, format='png')
#             buf.seek(0)
#             img = PIL.Image.open(buf)
#             img_data = io.BytesIO()
#             img.save(img_data, format="PNG")
#             img_uri = "data:image/png;base64," + base64.b64encode(img_data.getvalue()).decode()
            
#             # Close the Matplotlib figure
#             plt.close()

#             # Add the image to the Plotly figure using the data URI as the 'source' property
#             fig.add_layout_image(
#                 x=0,
#                 y=0.25,
#                 xref="paper",
#                 yref="paper",
#                 xanchor="left",
#                 yanchor="bottom",
#                 source=img_uri,
#                 opacity=1,
#                 layer="below"
#             )

# # Create a dropdown widget with topic options
# topic_dropdown = widgets.Dropdown(
#     options=pivoted_data['topic_list'].unique(),
#     description='Select Topic:',
# )

# # Create an interactive widget to select a topic and update the plot
# interactive(update_plot, selected_topic=topic_dropdown)

# # Show the initial Plotly figure
# display.display(fig)
